# Amplify anomalies using ENSO phases

In our experiments we amplify anomalies using El Niño-Southern Oscillation (ENSO) phases. So we have created a pair of auxiliary functions to do this. First we import `obrero` and the experimental `enso` module:

In [2]:
# small hack to be able to import module without install
import os
import sys
sys.path.append(os.getcwd() + '/../')

import obrero
from obrero.experimental import enso

Now we read the sea surface temperature (SST) data we wan to amplify following ENSO phases:

In [3]:
# filename
fname = 'data/sst_ctl.nc'

# read as datarray
da = obrero.read_nc(fname, 'tosbcs')

We will amplify anomalies in SST in order to obtain greater ONI values (which should mean stronger ENSO events). So we will get anomalies and climatology using `get_anomalies()` function in `obrero.analysis` module:

In [4]:
an, ac = obrero.get_anomalies(da, return_clim=True)

Notice we have used the `return_clim` keyword in order to save some lines in which we compute climatology. Also there is no need to compute it twice, since it is already being calculated for the anomalies. Now we need the ONI values:

In [5]:
oni = enso.get_oni(years=range(2005, 2011))

And to get amplified anomalies following ENSO we use function `amplify_with_enso()`. This function needs an anomalies array, an ONI data frame, a factor to amplify values and an optional bounds argument, in case the user only wants to amplify values in a specific region. Here we will amplify it in a rectangular area:

In [6]:
anamp = enso.amplify_with_enso(an, oni, 1.5, [100, 300, 20, -20])

And now we add these anomalies to the climatology to get the absolute values once again, but now amplified. To do this we have the function `add_climatology()` in the `obrero.analysis` module:

In [7]:
da_amp = obrero.add_climatology(anamp, ac)

If we go [here](https://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_v5.php), we can see that January of 2005 should have been amplified, however since we have sliced our ONI to start at January 2005, the ENSO finder (`obrero.experimental.enso.enso_finder()`) function will not be able to get the 5 month streak it needs to establish the El Niño. So the first month when we can see amplification working is in November 2005 when there was a La Niña. Let's slice our data arrays to the rectangular region we amplified and use the annual cycle to compute anomalies. Then we can see how the anomaly for some arbitrary grid cell inside the region has been amplified by a factor of 1.5:

In [8]:
# get rectangular regions
ac_reg = obrero.get_rectreg(ac, [100, 300, 20, -20])
orig = obrero.get_rectreg(da, [100, 300, 20, -20])
amp = obrero.get_rectreg(da_amp, [100, 300, 20, -20])

# print anomalies for December (11th position) for gridpoint (0,0)
print("Original anomaly:", orig.values[11, 0, 0] - ac_reg.values[11, 0, 0])
print("Amplified anomaly:", amp.values[11, 0, 0] - ac_reg.values[11, 0, 0])

Original anomaly: -0.40902209617973995
Amplified anomaly: -0.6135331442696383


So we can see that now the anomaly in the amplified array is 1.5 times the original value. Hopefully it is clear what `amplify_with_enso()` does.